In [25]:
import tensorflow.keras as keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import tensorflow.keras.callbacks as callbacks
from tensorflow.keras.utils import plot_model 


from nltk import word_tokenize, sent_tokenize
from tqdm import tqdm 
import numpy as np
import re
import string

In [2]:
path_to_file = keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
with open(path_to_file) as f:
    corpus = f.read().splitlines()
len(corpus)

40000

### Preprocessing 

In [4]:
punctuation_string = '!"\\\\#\\$%\\&\\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~'

In [5]:
def preprocess(corpus):
    sentences = sent_tokenize(' '.join(w for w in corpus))
    sentences = [sent.lower() for sent in sentences]
    sentences = [sent.decode() for sent in 
                 tf.strings.regex_replace(sentences,f'[{punctuation_string}]', '').numpy()]
    
    tokens = np.array([np.array(word_tokenize(sent), dtype='object') for sent in sentences], dtype='object')
    
    # return tokenized sentences
    return tokens

In [6]:
tokens_list = preprocess(corpus)

In [7]:
tokens_list.shape

(12460,)

In [8]:
tokens_list[:10]

array([array(['first', 'citizen', 'before', 'we', 'proceed', 'any', 'further',
       'hear', 'me', 'speak'], dtype=object),
       array(['all', 'speak', 'speak'], dtype=object),
       array(['first', 'citizen', 'you', 'are', 'all', 'resolved', 'rather',
       'to', 'die', 'than', 'to', 'famish'], dtype=object),
       array(['all', 'resolved'], dtype=object),
       array(['resolved'], dtype=object),
       array(['first', 'citizen', 'first', 'you', 'know', 'caius', 'marcius',
       'is', 'chief', 'enemy', 'to', 'the', 'people'], dtype=object),
       array(['all', 'we', "know't", 'we', "know't"], dtype=object),
       array(['first', 'citizen', 'let', 'us', 'kill', 'him', 'and', 'we', "'ll",
       'have', 'corn', 'at', 'our', 'own', 'price'], dtype=object),
       array(["is't", 'a', 'verdict'], dtype=object),
       array(['all', 'no', 'more', 'talking', 'o', "n't", 'let', 'it', 'be',
       'done', 'away', 'away'], dtype=object)], dtype=object)

In [9]:
def build_vocabulary(tokens_list):
    stoi = {}
    itos = {}
    idx = 0
    
    stoi['<pad>'] = idx
    itos[idx] = '<pad>'
    idx += 1
    
    for tokens in tqdm(tokens_list):
        for word in tokens:
            if word not in stoi:
                stoi[word] = idx
                itos[idx] = word
                idx += 1
    return stoi, itos

In [10]:
# remove '
tokens_list
stoi, itos = build_vocabulary(tokens_list)

100%|██████████| 12460/12460 [00:00<00:00, 214126.62it/s]


In [11]:
numericalized = [[stoi[w] for w in sent] for sent in tokens_list]

In [12]:
vocab_size = len(stoi)
embed_size = 100
window_size = 2

print(f"Vocab Size  : {vocab_size}")
print(f"Vocab Sample: {list(stoi.items())[:5]}")

Vocab Size  : 12483
Vocab Sample: [('<pad>', 0), ('first', 1), ('citizen', 2), ('before', 3), ('we', 4)]


In [13]:
def generate_context_target_pairs(corpus=numericalized, window_size=2, vocab_size=vocab_size):
    
    context_length = window_size * 2
    
    for sentence in corpus:
        sentence_length = len(sentence)
        
        for index, word in enumerate(sentence):
            context_words = []
            label_word = []
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([sentence[i] for i in range(start, end)
                                 if 0 <= i < sentence_length and i != index])
            label_word.append(word)
            
            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, vocab_size)
            yield(x[0], y[0])

In [16]:
i = 0
for x, y in generate_context_target_pairs(numericalized, 2, vocab_size):
    if 0 not in x:
        if i == 10: break
        i+=1

        print(f'Target: {itos[np.argmax(y)]}, Context: {[itos[w] for w in x]}')

Target: before, Context: ['first', 'citizen', 'we', 'proceed']
Target: we, Context: ['citizen', 'before', 'proceed', 'any']
Target: proceed, Context: ['before', 'we', 'any', 'further']
Target: any, Context: ['we', 'proceed', 'further', 'hear']
Target: further, Context: ['proceed', 'any', 'hear', 'me']
Target: hear, Context: ['any', 'further', 'me', 'speak']
Target: you, Context: ['first', 'citizen', 'are', 'all']
Target: are, Context: ['citizen', 'you', 'all', 'resolved']
Target: all, Context: ['you', 'are', 'resolved', 'rather']
Target: resolved, Context: ['are', 'all', 'rather', 'to']


In [17]:
dataset = tf.data.Dataset.from_generator(generate_context_target_pairs, (tf.int16, tf.int16)).batch(100000)

In [18]:
for batch, (x, y) in enumerate(dataset):
  break
print("batch: ", batch)
x.shape, y.shape

batch:  0


(TensorShape([100000, 4]), TensorShape([100000, 12483]))

In [30]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2),
    Lambda(lambda x: keras.backend.mean(x, axis=1), output_shape=(embed_size)),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            1248300   
_________________________________________________________________
lambda_1 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 12483)             1260783   
Total params: 2,509,083
Trainable params: 2,509,083
Non-trainable params: 0
_________________________________________________________________


In [32]:
plot_model(model,show_shapes=True, show_layer_names=True, rankdir='TB', expand_nested=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [34]:
google_drive_path = ''

In [35]:
early_stopping_cb = callbacks.EarlyStopping(patience=10, verbose=True)
checkpoint_cb = callbacks.ModelCheckpoint(google_drive_path, verbose=True, save_best_only=True)
tensorboard_callback = callbacks.TensorBoard(log_dir="logs")

callbacks_list = [early_stopping_cb, checkpoint_cb, tensorboard_callback]

history = model.fit(x, y, epochs=1, callbacks=callbacks_list)

3125/3125 [==============================] - 64s 20ms/step - loss: 7.4852 - accuracy: 0.0333
